In [1]:
import numpy as np
import matplotlib.pyplot as plt
from image_graph_util import img_conversion_v7
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from emnist import extract_training_samples,extract_test_samples
from emnist import list_datasets
from sklearn.utils import shuffle
import cv2
import glob

2023-07-25 12:53:46.669633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-25 12:53:46.669678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
img=img_conversion_v7.image_convert()


patch=2

In [3]:
ds, info = tfds.load('malaria', split='train', shuffle_files=True, with_info=True)

2023-07-25 12:54:18.437841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/user/home/sahaj432/.local/lib/python3.9/site-packages/cv2/../../lib64:/cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-25 12:54:18.457436: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-25 12:54:18.457526: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0188): /proc/driver/nvidia/version does not exist
2023-07-25 12:54:18.458437: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other opera

In [4]:
train_images = []
train_labels = []

for example in ds:
    train_images.append(example['image'].numpy())
    train_labels.append(example['label'].numpy())

In [5]:
print(len(train_images))
print(len(train_labels))

27558
27558


In [6]:
images=np.array([cv2.resize(i,(100,100), interpolation= cv2.INTER_CUBIC) for i in train_images])


In [7]:
images = [i/255 for i in images]
labels = np.array(train_labels)

In [8]:
images=np.array(images,dtype="float16")

In [9]:
images.shape

(27558, 100, 100, 3)

In [10]:
train,test,train_lab,test_lab=train_test_split(images,labels,test_size=0.30,random_state=42)


In [11]:
print(train.shape)
print(test.shape)
print(train_lab.shape)
print(test_lab.shape)

(19290, 100, 100, 3)
(8268, 100, 100, 3)
(19290,)
(8268,)


In [12]:
regular_train=train
reg_train_label=train_lab

In [13]:
regular_val,regular_test,reg_val_label,reg_test_label=train_test_split(test,test_lab,test_size=0.50,random_state=42)


In [14]:
print(regular_train.shape)
print(regular_val.shape)
print(regular_test.shape)



(19290, 100, 100, 3)
(4134, 100, 100, 3)
(4134, 100, 100, 3)


In [15]:
reg_train_patch=img.to_patcharray(regular_train[:],patch_size=(patch,patch))
reg_val_patch=img.to_patcharray(regular_val[:],patch_size=(patch,patch))
reg_test_patch=img.to_patcharray(regular_test[:],patch_size=(patch,patch))



In [16]:
print(reg_train_patch.shape)
print(reg_val_patch.shape)
print(reg_test_patch.shape)

(19290, 3, 2500, 2, 2)
(4134, 3, 2500, 2, 2)
(4134, 3, 2500, 2, 2)


In [17]:
from collections import Counter
#print((Counter(y_val)))
print(np.sort(list(Counter(reg_train_label).keys())))
classes=len(np.sort(list(Counter(reg_train_label).keys())))

[0 1]


# Gershgorien image analysis

In [18]:
import networkx as nx



def compute_lap_to_gersh(df_in): #compute_lap_to_gersh
    adj_ary=np.ones(reg_train_patch.shape[-2]*reg_train_patch.shape[-1])-np.eye(reg_train_patch.shape[-2]*reg_train_patch.shape[-1])

    gersh_c=np.tile(np.sum(adj_ary,axis=0),(reg_train_patch.shape[1],reg_train_patch.shape[2]))

    edge=np.array(np.where(adj_ary==1))
    ed0=edge[0]
    ed1=edge[1]
    no_img,no_ch, no_patch,patch_x,patch_y=df_in.shape
    temp=[]
    for i in range(no_img):
        i_ary = np.eye(patch_x*patch_y)
        i_ary4d=i_ary[np.newaxis,np.newaxis,:,:]
        weighted_adj=np.zeros((no_ch, no_patch,patch_x*patch_y,patch_x*patch_y))
        df=df_in[i].reshape(no_ch,no_patch,-1)
        pixel1=df[:,:,ed0]
        pixel2=df[:,:,ed1]  
        edu_distance=np.sqrt((pixel1-pixel2)**2)+ 10**(-8)
        weighted_adj[:,:,ed0,ed1]=edu_distance
        sum_r=np.sum(weighted_adj,axis=2)
        gersh_r=sum_r.reshape(no_ch,-1)
        gersh_total=np.vstack((gersh_r[0],gersh_c[0],gersh_r[1],gersh_c[1],gersh_r[2],gersh_c[2]))
        gersh_total=gersh_total.reshape(no_ch,2,gersh_c.shape[-1])
        temp.append(gersh_total)
    return temp



In [19]:
from multiprocessing import Pool
import multiprocessing



num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [20]:
train_patch1=np.array_split(reg_train_patch[:], num_cores)
val_patch1=np.array_split(reg_val_patch[:], num_cores)
test_patch1=np.array_split(reg_test_patch[:], num_cores)

In [21]:
print(len(train_patch1))
print(len(val_patch1))
print(len(test_patch1))

48
48
48


In [22]:
num_matrices, num_channels, num_patches, height, width = train_patch1[0].shape

In [23]:
with Pool(processes=num_cores) as pool:
        # Define a list of numbers
    train_p = train_patch1
    
        # Apply the square function to each number using the pool
    g_train=pool.map(compute_lap_to_gersh,train_p)
pool.close()


    
        #results = pool.map(calculate_visibility, numbers)


In [24]:
temp1=[]
for submatrix in g_train:
    temp1.extend(submatrix)


In [25]:
gersh_train= np.array([np.asarray(submatrix) for submatrix in temp1])


In [26]:
print("d")

d


In [27]:
with Pool(processes=num_cores) as pool1:
        # Define a list of numbers
    val_p = val_patch1
    
        # Apply the square function to each number using the pool
    g_val=pool1.map(compute_lap_to_gersh,val_patch1)    
pool1.close()



In [28]:
temp2=[]
for submatrix in g_val:
    temp2.extend(submatrix)


In [29]:
gersh_val= np.array([np.asarray(submatrix) for submatrix in temp2])


In [30]:
print("d")

d


In [31]:
with Pool(processes=num_cores) as pool2:
        # Define a list of numbers
    test_p = test_patch1
    
        # Apply the square function to each number using the pool
    g_test=pool2.map(compute_lap_to_gersh,test_patch1) 
pool2.close()


In [32]:
temp3=[]
for submatrix in g_test:
    temp3.extend(submatrix)


In [33]:
gersh_test= np.array([np.asarray(submatrix) for submatrix in temp3])


gersh_train=np.array(img1.to_gridlaparray(train_patch))
gersh_val=np.array(img1.to_gridlaparray(val_patch))
gersh_test=np.array(img1.to_gridlaparray(test_patch))

In [34]:
print(gersh_train.shape)
print(gersh_val.shape)
print(gersh_test.shape)

(19290, 3, 2, 10000)
(4134, 3, 2, 10000)
(4134, 3, 2, 10000)


In [35]:
rgersh_train=np.moveaxis(gersh_train,1,-1)
rgersh_val=np.moveaxis(gersh_val,1,-1)
rgersh_test=np.moveaxis(gersh_test,1,-1)

In [36]:
print(rgersh_train.shape)
print(rgersh_val.shape)
print(rgersh_test.shape)

(19290, 2, 10000, 3)
(4134, 2, 10000, 3)
(4134, 2, 10000, 3)


In [37]:
x_train=rgersh_train.reshape(rgersh_train.shape[0],-1)
x_val=rgersh_val.reshape(rgersh_val.shape[0],-1)
x_test=rgersh_test.reshape(rgersh_test.shape[0],-1)

In [38]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(19290, 60000)
(4134, 60000)
(4134, 60000)


In [39]:
input_shape = (x_train.shape[1],1)
num_classes = classes
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model2.add(tf.keras.layers.AveragePooling1D())
model2.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model2.add(tf.keras.layers.AveragePooling1D())
model2.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model2.add(tf.keras.layers.AveragePooling1D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(512, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.1))
model2.add(tf.keras.layers.Dense(512, activation='relu'))
model2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model2.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [40]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60000, 32)         128       
                                                                 
 average_pooling1d (AverageP  (None, 30000, 32)        0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 30000, 32)         3104      
                                                                 
 average_pooling1d_1 (Averag  (None, 15000, 32)        0         
 ePooling1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 15000, 32)         3104      
                                                                 
 average_pooling1d_2 (Averag  (None, 7500, 32)         0

In [41]:
history_log=model2.fit(x_train, reg_train_label, epochs=10,verbose=1,validation_data=(x_val, reg_val_label))


Epoch 1/10
603/603 [==============================] - 289s 478ms/step - loss: 0.6568 - acc: 0.6976 - val_loss: 0.2280 - val_acc: 0.9231
Epoch 2/10
603/603 [==============================] - 291s 482ms/step - loss: 0.1895 - acc: 0.9348 - val_loss: 0.2397 - val_acc: 0.9190
Epoch 3/10
603/603 [==============================] - 290s 481ms/step - loss: 0.1139 - acc: 0.9614 - val_loss: 0.2006 - val_acc: 0.9357
Epoch 4/10
603/603 [==============================] - 288s 478ms/step - loss: 0.0456 - acc: 0.9850 - val_loss: 0.3166 - val_acc: 0.9238
Epoch 5/10
603/603 [==============================] - 292s 485ms/step - loss: 0.0198 - acc: 0.9937 - val_loss: 0.4054 - val_acc: 0.9330
Epoch 6/10
603/603 [==============================] - 301s 499ms/step - loss: 0.0109 - acc: 0.9968 - val_loss: 0.3989 - val_acc: 0.9325
Epoch 7/10
603/603 [==============================] - 297s 493ms/step - loss: 0.0112 - acc: 0.9967 - val_loss: 0.4421 - val_acc: 0.9299
Epoch 8/10
603/603 [============================

In [42]:
loss, accuracy = model2.evaluate(x_test, reg_test_label)
print(f' Model loss on the test set: {loss}')
print(f' Model accuracy on the test set: {100*accuracy}')

130/130 [==============================] - 9s 70ms/step - loss: 0.6124 - acc: 0.9267
 Model loss on the test set: 0.6123838424682617
 Model accuracy on the test set: 92.67053604125977


In [43]:
predictions = model2.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

In [44]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(reg_test_label,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1924  110]
 [ 193 1907]]


In [45]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [46]:
print(accuracy_score(reg_test_label, pred_arg))
print(f1_score(reg_test_label,pred_arg,average='macro'))
print(recall_score(reg_test_label, pred_arg, average="weighted"))
print(precision_score(reg_test_label, pred_arg, average="weighted"))

0.9267053701015965
0.926704130630515
0.9267053701015965
0.9274408116415915
